In [ ]:
import numpy as np
import pandas as pd
import scipy.io as sio # for loading MATLAB files
import matplotlib.pyplot as plt
import seaborn as sns
sns.set() #sets the matplotlib style to seaborn style

from scipy.io import loadmat 
from scipy.ndimage import convolve1d
from scipy.signal import butter, filtfilt, iirnotch, sosfiltfilt, welch

import os

In [ ]:
raw_data_path = 'data/pers_data/raw/' # path to raw data
processed_data_path = 'data/pers_data/cleaned/' # path to processed data

# Create directories if they do not exist
os.makedirs(raw_data_path, exist_ok=True)
os.makedirs(processed_data_path, exist_ok=True)

In [ ]:
for root, d_names, f_names in os.walk(raw_data_path):
    print('File names:', f_names)

In [ ]:
for file in f_names:
    raw_data = pd.read_csv(raw_data_path + file)
    print(raw_data.head())

In [ ]:
def hampel_filter(series, window_size=3, n_sigmas=3):
    """Applies Hampel filter to a 1D pandas Series."""
    L = 1.4826  # scale factor for Gaussian distribution
    rolling_median = series.rolling(window=2 * window_size + 1, center=True).median()
    diff = np.abs(series - rolling_median)
    mad = L * diff.rolling(window=2 * window_size + 1, center=True).median()
    
    # Identify outliers
    outlier_idx = diff > n_sigmas * mad
    filtered = series.copy()
    filtered[outlier_idx] = rolling_median[outlier_idx]
    return filtered

In [ ]:
def clean_data(raw_data):
    cleaned_data = raw_data.copy()
    
    # Remove the first row
    cleaned_data = cleaned_data.drop(index=0)

    # Number of channels
    num_channels = 2

    # Compute the average sampling frequency and jitter
    sampling_freq = 1 / np.mean(np.diff(cleaned_data["Timestamp"]))
    jitter = np.mean(np.diff(cleaned_data["Timestamp"]))
    print(f"Sampling Frequency: {sampling_freq} Hz")
    print(f"Jitter: {jitter} seconds")

    # # Resample the data to a fixed sampling frequency (e.g., 1000 Hz)
    # target_rate = 2000  # Hz
    # start_time = cleaned_data['Timestamp'].iloc[0]
    # end_time = cleaned_data['Timestamp'].iloc[-1]
    # uniform_time = np.arange(start_time, end_time, 1 / target_rate)
    # # Interpolate both channels to the new time grid
    # resampled_data = pd.DataFrame({
    #     'Timestamp': uniform_time,
    #     })
    # for i in range(num_channels):
    #     resampled_data["Channel{}".format(i+1)] = np.interp(uniform_time, cleaned_data["Timestamp"], cleaned_data["Channel{}".format(i+1)])
    # # Still misses the Action columns...


    for i in range(num_channels): # nb of channels
        channel = cleaned_data["Channel{}".format(i+1)]

        # # Normalize the channel
        # channel = (channel - np.mean(channel))/np.std(channel)

        # Remove DC offset
        channel = (channel - np.mean(channel))

        # Filter 50Hz noise using a Hampel filter
        channel = hampel_filter(channel)

        cleaned_data["Channel{}".format(i+1)] = channel
    
    return cleaned_data

In [ ]:
for file in f_names:
    raw_data = pd.read_csv(raw_data_path + file)
    cleaned_data = clean_data(raw_data)
    print(cleaned_data.head())
    # break
    cleaned_data.to_csv(processed_data_path + "cleaned_" + file, index=False)

In [ ]:
start_id = cleaned_data[(cleaned_data["Action1"] == "Rest") & (cleaned_data["Action2"] == "Fist")].index[0]
end_id = cleaned_data[(cleaned_data["Action1"] == "Fist") & (cleaned_data["Action2"] == "Rest")].index[-1]
window_df = cleaned_data.loc[start_id:end_id]

# plt.figure(figsize=(12, 4))
# plt.plot(window_df['Timestamp'], window_df['Channel1'], label='Channel1')
# plt.plot(window_df['Timestamp'], window_df['Channel2'], label='Channel2')
# plt.title("First 'Fist' Action Window")
# plt.xlabel("Time (s)")
# plt.ylabel("Signal")
# plt.legend()
# plt.grid()
# plt.tight_layout()
# plt.show()

fig, ax = plt.subplots(2, figsize=(12, 6), constrained_layout=True, sharex=True, sharey=True)
ax = ax.ravel()

#2 EMG channels
for channel_idx in range(2): 
    ax[channel_idx].plot(window_df['Timestamp'], window_df['Channel{}'.format(channel_idx+1)])
    ax[channel_idx].set_title(f"Channel {channel_idx+1}")
plt.suptitle("Rectified EMG signal for restimulus 1")